In [ ]:
# Menginstal library transformers dan lainnya
!pip install transformers[sentencepiece] datasets evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=53227f132cf9aff73ae570eb06be98cfc0ccbe4e46348d2f9b28f5dc44c440fa
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
from transformers import pipeline

# Menyiapkan Teks Berita dalam Bahasa Indonesia
teks_indo = """
Kecerdasan Buatan (AI) telah mengalami perkembangan yang sangat pesat dalam beberapa tahun terakhir.
Teknologi ini mulai diterapkan di berbagai sektor, mulai dari kesehatan, pendidikan, hingga transportasi.
Salah satu terobosan terbesar adalah munculnya Large Language Models (LLM) yang mampu memproses dan menghasilkan teks layaknya manusia.
Meskipun memberikan banyak manfaat, seperti efisiensi kerja dan inovasi baru, perkembangan AI juga memunculkan kekhawatiran terkait etika dan masa depan pekerjaan.
Banyak ahli menyarankan perlunya regulasi yang ketat untuk memastikan AI digunakan demi kebaikan umat manusia, bukan sebaliknya.
Pemerintah Indonesia sendiri mulai merancang strategi nasional kecerdasan buatan untuk menghadapi era digital ini.
"""

# Inisialisasi Pipeline
# Kita meminta tugas "summarization" dengan model spesifik bahasa Indonesia
summarizer = pipeline("summarization", model="cahya/bert2bert-indonesian-summarization")

# Peringkasan
hasil_pipeline = summarizer(teks_indo)

print("--- Hasil Ringkasan (Pipeline) ---")
print(hasil_pipeline[0]['summary_text'])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/999M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/999M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


--- Hasil Ringkasan (Pipeline) ---
perkembangan ai memunculkan kekhawatiran terkait etika dan masa depan pekerjaan. banyak ahli menyarankan perlunya regulasi yang ketat untuk memastikan ai digunakan demi kebaikan umat manusia.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_checkpoint = "cahya/bert2bert-indonesian-summarization"

# Memuat Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Memuat Model
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
# Mengubah teks menjadi tensor
input_tokens = tokenizer(teks_indo, return_tensors="pt", padding="max_length", truncation=True, max_length=512)

print("Bentuk Data (Shape):", input_tokens["input_ids"].shape)
print("5 Token Pertama (Angka):", input_tokens["input_ids"][0][:5])

Bentuk Data (Shape): torch.Size([1, 512])
5 Token Pertama (Angka): tensor([    3, 12925,  4994,    11, 10239])


In [ ]:
# Menjalankan proses generasi (Inference)
summary_ids = model.generate(
    input_tokens["input_ids"],
    max_length=100,      # Batas maksimal panjang ringkasan
    min_length=20,       # Batas minimal ringkasan
    num_beams=5,         # Menggunakan 5 jalur Beam Search untuk mencari kalimat terbaik
    early_stopping=True,
    no_repeat_ngram_size=2 # Mencegah model mengulang frasa yang sama (supaya tidak repetitif)
)

# Decoding
ringkasan_manual = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("--- Hasil Ringkasan (Manual dengan Beam Search) ---")
print(ringkasan_manual)

--- Hasil Ringkasan (Manual dengan Beam Search) ---
perkembangan ai memunculkan kekhawatiran terkait etika dan masa depan pekerjaan. banyak ahli menyarankan perlunya regulasi yang ketat untuk memastikan ai digunakan demi kebaikan umat manusia.


In [ ]:
import evaluate

# Menyiapkan Metrik
rouge = evaluate.load('rouge')

# Menyiapkan Kunci Jawaban
referensi = ["Kecerdasan Buatan (AI) berkembang pesat dan membawa manfaat seperti efisiensi, namun memunculkan kekhawatiran etika sehingga perlu regulasi."]

# Menghitung Skor
skor = rouge.compute(predictions=[ringkasan_manual], references=referensi)

print("Skor Evaluasi ROUGE:")
print(skor)

Skor Evaluasi ROUGE:
{'rouge1': np.float64(0.28571428571428564), 'rouge2': np.float64(0.05), 'rougeL': np.float64(0.2380952380952381), 'rougeLsum': np.float64(0.2380952380952381)}


In [ ]:
# Ganti Checkpoint ke Model Bahasa Inggris (T5-Small)
model_eng_checkpoint = "t5-small"
tokenizer_eng = AutoTokenizer.from_pretrained(model_eng_checkpoint)
model_eng = AutoModelForSeq2SeqLM.from_pretrained(model_eng_checkpoint)

# Teks Bahasa Inggris
text_eng = """
The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.
It is named after the engineer Gustave Eiffel, whose company designed and built the tower.
Locally nicknamed "La Dame de Fer" (French for "The Iron Lady"), it was constructed from 1887 to 1889
as the centerpiece of the 1889 World's Fair.
"""

# Pada T5, kita harus memberi prefix "summarize: " di awal teks
input_text_formatted = "summarize: " + text_eng

# Tokenize -> Generate -> Decode
inputs = tokenizer_eng(input_text_formatted, return_tensors="pt", max_length=512, truncation=True)

summary_ids_eng = model_eng.generate(
    inputs["input_ids"],
    max_length=50,
    min_length=10,
    do_sample=True,
    temperature=0.7
)

summary_eng = tokenizer_eng.decode(summary_ids_eng[0], skip_special_tokens=True)

print("--- English Summary (T5 Model) ---")
print(summary_eng)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

--- English Summary (T5 Model) ---
the Eiffel Tower is named after the engineer Gustave Eiffel. it was the centerpiece of the 1889 world's fair in 1889.
